In [29]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions 
import time

data=[]
divs=[]
executable_path = '/Users/jason/Documents/BIA study/Web Mining/Resources/Web_Scraping/chromedriver'
driver = webdriver.Chrome(executable_path=executable_path)
driver.implicitly_wait(10)
driver.get("https://www.groupon.com/deals/artistic-image-salon-spa-2")

#see all reviews

try:
    all_reviews=driver.find_element_by_css_selector("#all-tips-link")
    all_reviews.click()
except:
    all_reviews=driver.find_element_by_css_selector("#nothx")
    all_reviews.click()
    time.sleep(2)
    all_reviews=driver.find_element_by_css_selector("#all-tips-link")
    all_reviews.click()


#get the number of pages
#elements return list, add [].text to extract content
pages=driver.find_elements_by_css_selector("#all-tips div.tip-pagination ul.tip-pagination-links a.tip-pagination-link.number")
page_num=int(pages[-1].text)

#get the content of the first page
soup = BeautifulSoup(driver.page_source, 'html.parser')
divs.append(soup.select('#all-tips div.tip-widget-tips div.tip-body'))

#get the content of the rest pages by loop
for i in range(page_num-1):
    nextp=driver.find_element_by_css_selector("#all-tips div.tip-pagination ul.tip-pagination-links a.tip-pagination-link.next-link ")
    nextp.click()
    time.sleep(2)        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs.append(soup.select('#all-tips div.tip-widget-tips div.tip-body'))
    
driver.quit() 
    
for i in range(page_num):
    for idx,div in enumerate(divs[i]):

        date=None
        description=None
        user=None
        rating=None
        rating_count=None
        reviews_count=None
        #collect date
        date1=div.select('div.twelve.columns.ratings-date span.reviewer-reviewed-date')
        if date1!=[]:
            date=date1[0].get_text()
        #collect reviews description
        description1=div.select('div.twelve.columns.tip-text')
        if description1!=[]:
            description=description1[0].get_text().strip()
        #collect user name
        user1=div.select('span.tips-reviewer-name')
        if user1!=[]:
            user=user1[0].get_text()
        #collect rating star,count how many full-star valued class span to get rate
        rating1=div.select('div.twelve.columns.ratings-date ul.product-histogram-rating span.full-star')
        rating=len(rating1)
        #collect user's rating count
        rating_count1=div.select('div.user-stats span.reviewer-review-count')
        if rating_count1!=[]:
            rating_count=int(rating_count1[0].get_text().split()[0])
        #collect user's reviews count
        reviews_count1=div.select('div.user-stats span.reviewer-review-count')
        if reviews_count1!=[]:
            reviews_count=int(reviews_count1[1].get_text().split()[0])        
        #append into tuple       
        data.append((date,description,user,rating,rating_count,reviews_count))

    
print(len(data))



145


In [30]:
import csv
import pandas as pd 
#convert the data into the csv file
data_resource=list(data)
data_resource=pd.DataFrame(data_resource,columns=["Date","Description","Name","Reting","rating_count","reviews_count"])
data_resource.to_csv("/Users/jason/Documents/BIA study/Web Mining/Final Project/Data mining/data_resource4.csv", sep=',',index=False)
